In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Defina o caminho para a sua pasta no Google Drive
base_path = '/content/drive/MyDrive/Colab Notebooks/Olist/' # Altere para o seu caminho real

# Carregar a tabela de reviews
reviews_df = pd.read_csv(f'{base_path}olist_order_reviews_dataset.csv')

# Opcional: Carregar outras tabelas se precisar de merges no Colab
# orders_df = pd.read_csv(f'{base_path}olist_orders_dataset.csv')
# products_df = pd.read_csv(f'{base_path}olist_products_dataset.csv')

print(reviews_df.head())
print(reviews_df.info())

                          review_id                          order_id  \
0  7bc2406110b926393aa56f80a40eba40  73fc7af87114b39712e6da79b0a377eb   
1  80e641a11e56f04c1ad469d5645fdfde  a548910a1c6147796b98fdf73dbeba33   
2  228ce5500dc1d8e020d8d1322874b6f0  f9e4b658b201a9f2ecdecbb34bed034b   
3  e64fb393e7b32834bb789ff8bb30750e  658677c97b385a9be170737859d3511b   
4  f7c4243c7fe1938f181bec41a392bdeb  8e6bfb81e283fa7e4f11123a3fb894f1   

   review_score review_comment_title  \
0             4                  NaN   
1             5                  NaN   
2             5                  NaN   
3             5                  NaN   
4             5                  NaN   

                              review_comment_message review_creation_date  \
0                                                NaN  2018-01-18 00:00:00   
1                                                NaN  2018-03-10 00:00:00   
2                                                NaN  2018-02-17 00:00:00   
3           

In [ ]:
# Cell 1: Install Specific Versions (Run this FIRST after Factory Reset)
# Ensure these run without errors.

!pip install torch==2.0.1+cu118 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers==4.26.1
!pip install flair==0.11.3 # This version of flair is generally compatible
!pip install pandas
!pip install regex
!pip install tqdm
!pip install --upgrade pip

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:
# ==============================================================================
# CÉLULA 1: INSTALAÇÃO DE BIBLIOTECAS
# ==============================================================================
# Usamos o -q para uma instalação mais "silenciosa"
print("🚀 Iniciando instalação de bibliotecas...")
!pip install -q torch==2.0.1+cu118 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q transformers==4.26.1 flair==0.11.3 pandas regex tqdm
print("✅ Bibliotecas instaladas com sucesso!")

# ==============================================================================
# CÉLULA 2: IMPORTAÇÕES E CONFIGURAÇÃO DO AMBIENTE
# ==============================================================================
import pandas as pd
import re
from tqdm.notebook import tqdm
from google.colab import drive
from flair.models import TextClassifier
from flair.data import Sentence

# Habilitar o .progress_apply() do pandas para barras de progresso
tqdm.pandas()

print("✅ Módulos importados e ambiente configurado.")

# ==============================================================================
# CÉLULA 3: CARREGAMENTO DOS DADOS DO GOOGLE DRIVE
# ==============================================================================
print("\n🔄 Montando Google Drive...")
try:
    drive.mount('/content/drive')
except Exception as e:
    print(f"🚨 Erro ao montar o Google Drive: {e}")

# --- ATENÇÃO: SUBSTITUA PELO SEU CAMINHO ---
# Certifique-se de que este é o caminho correto para a pasta que contém o seu CSV.
base_path = '/content/drive/MyDrive/Colab Notebooks/Olist/'
# -------------------------------------------

try:
    print(f"🔄 Carregando dataset de '{base_path}olist_order_reviews_dataset.csv'...")
    reviews_df = pd.read_csv(f'{base_path}olist_order_reviews_dataset.csv')
    print("✅ DataFrame 'reviews_df' carregado com sucesso!")
    print("\n📋 Informações do DataFrame:")
    reviews_df.info(show_counts=True)
    print("\n👀 Amostra dos dados:")
    print(reviews_df.head())
except FileNotFoundError:
    print(f"❌ ERRO CRÍTICO: O arquivo 'olist_order_reviews_dataset.csv' não foi encontrado em '{base_path}'.")
    print("👉 Verifique o caminho em `base_path` e se o arquivo está no seu Google Drive.")
    # Interrompe a execução se o arquivo principal não for encontrado
    exit()

# ==============================================================================
# CÉLULA 4: PRÉ-PROCESSAMENTO E LIMPEZA DOS DADOS
# ==============================================================================
print("\n🧹 Iniciando pré-processamento e limpeza de texto...")

# 1. Tratar valores nulos (NaN) nas colunas de texto
reviews_df['review_comment_title'] = reviews_df['review_comment_title'].fillna('')
reviews_df['review_comment_message'] = reviews_df['review_comment_message'].fillna('')

# 2. Concatenar título e mensagem para criar um texto de review completo
#    Adicionamos um ponto final ao título para uma separação semântica melhor.
reviews_df['review_full_text'] = reviews_df['review_comment_title'].str.strip() + '. ' + reviews_df['review_comment_message'].str.strip()

# 3. Função de limpeza de texto
#    - Converte para minúsculas
#    - Remove URLs
#    - Remove caracteres especiais (mantendo letras, números e espaços)
#    - Remove espaços extras
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text) # Remove URLs
    text = re.sub(r'[^a-z0-9\s]', '', text) # Remove caracteres não alfanuméricos
    text = re.sub(r'\s+', ' ', text).strip() # Remove espaços múltiplos
    return text

# 4. Aplicar a limpeza e criar uma nova coluna 'cleaned_text'
reviews_df['cleaned_text'] = reviews_df['review_full_text'].progress_apply(clean_text)

# 5. Filtrar apenas reviews que contêm texto após a limpeza
#    Isso evita processar strings vazias, economizando tempo.
#    O .copy() evita o SettingWithCopyWarning.
reviews_to_process_df = reviews_df[reviews_df['cleaned_text'] != ''].copy()

print(f"\n✅ Pré-processamento concluído.")
print(f"Total de reviews no dataset: {len(reviews_df)}")
print(f"Reviews com texto para análise: {len(reviews_to_process_df)}")
print("\n👀 Amostra do texto concatenado e limpo:")
print(reviews_to_process_df[['review_full_text', 'cleaned_text']].head())


# ==============================================================================
# CÉLULA 5: CARREGAMENTO DO MODELO DE SENTIMENTO E DIAGNÓSTICO
# ==============================================================================
print("\n🧠 Carregando o modelo de sentimento para Português...")

try:
    # Este é o modelo correto e robusto para português.
    classifier = TextClassifier.load('PORTULAN/XLMR_Sentiment_Portuguese')
    print("✅ Modelo 'PORTULAN/XLMR_Sentiment_Portuguese' carregado com sucesso.")
except Exception as e:
    print(f"❌ ERRO CRÍTICO: Falha ao carregar o modelo Flair: {e}")
    exit()

# --- DIAGNÓSTICO IMPORTANTE ---
# Vamos testar o modelo com algumas frases para ver exatamente quais rótulos ele retorna.
# Isso confirma o problema do código original.
print("\n🔍 Executando diagnóstico do modelo...")
test_sentences = [
    Sentence("Eu adorei o produto, a entrega foi muito rápida!"),
    Sentence("A qualidade é péssima, não recomendo."),
    Sentence("O produto chegou no prazo correto."),
]

classifier.predict(test_sentences)

for sentence in test_sentences:
    print(f'Frase: "{sentence.text}" -> Rótulo Previsto: {sentence.labels[0]}')

print("\n✨ Diagnóstico concluído. Note que os rótulos são 'Positivo', 'Negativo' e 'Neutro'.")


# ==============================================================================
# CÉLULA 6: ANÁLISE DE SENTIMENTO EFICIENTE (EM BATCH)
# ==============================================================================
# Para datasets grandes, processar um por um com .apply() é muito lento.
# A melhor abordagem é criar uma lista de Sentences e processá-los em lotes (batches).

print("\n🏃‍♂️ Iniciando a análise de sentimento em lote (isso pode levar vários minutos)...")

# --- PARÂMETRO PARA TESTE ---
# Se quiser testar com um subconjunto menor primeiro, descomente a linha abaixo.
# reviews_to_process_df = reviews_to_process_df.head(1000)
# -----------------------------

# 1. Criar a lista de objetos Sentence a partir da nossa coluna de texto limpo
sentences = [Sentence(text) for text in reviews_to_process_df['cleaned_text']]

# 2. Executar a predição em lote (mini-batch)
#    O mini_batch_size depende da memória RAM da sua GPU no Colab. 32 ou 64 é um bom começo.
classifier.predict(sentences, mini_batch_size=32)

# 3. Extrair os resultados (polaridade e classe) da lista de sentenças
sentiments = []
polarities = []

# Mapeamento dos rótulos para a polaridade
label_to_polarity = {
    'Positivo': 1,
    'Negativo': -1,
    'Neutro': 0
}

for sentence in tqdm(sentences, desc="Extraindo resultados"):
    if sentence.labels:
        label = sentence.labels[0]
        sentiment_class = label.value
        # A polaridade é a confiança do modelo multiplicada pelo valor do sentimento (+1, -1, 0)
        sentiment_polarity = label.score * label_to_polarity.get(sentiment_class, 0)

        sentiments.append(sentiment_class)
        polarities.append(sentiment_polarity)
    else:
        # Fallback caso alguma predição falhe (raro)
        sentiments.append("Não Avaliado")
        polarities.append(0.0)

# 4. Adicionar os resultados de volta ao DataFrame que foi processado
reviews_to_process_df['Sentiment_Class'] = sentiments
reviews_to_process_df['Sentiment_Polarity'] = polarities

print("\n✅ Análise de sentimento concluída!")
print("\n📊 Contagem das classes de sentimento:")
print(reviews_to_process_df['Sentiment_Class'].value_counts())
print("\n👀 Amostra dos resultados:")
print(reviews_to_process_df[['cleaned_text', 'Sentiment_Class', 'Sentiment_Polarity']].head())


# ==============================================================================
# CÉLULA 7: FINALIZAÇÃO E EXPORTAÇÃO DOS RESULTADOS
# ==============================================================================
print("\n💾 Finalizando e preparando para exportação...")

# 1. Juntar os resultados do dataframe processado de volta ao dataframe original
#    Isso garante que todos os reviews (incluindo os que não tinham texto)
#    estejam no arquivo final.
final_df = reviews_df.merge(
    reviews_to_process_df[['review_id', 'Sentiment_Class', 'Sentiment_Polarity']],
    on='review_id',
    how='left' # 'left' join para manter todos os reviews do dataframe original
)

# 2. Preencher os valores para reviews que não foram processados (sem texto)
final_df['Sentiment_Class'] = final_df['Sentiment_Class'].fillna('Não Avaliado')
final_df['Sentiment_Polarity'] = final_df['Sentiment_Polarity'].fillna(0.0)

# 3. Definir o nome do arquivo de saída
output_filename = 'olist_reviews_com_sentimento.csv'
output_path = f'{base_path}{output_filename}'

# 4. Exportar o DataFrame final para CSV
print(f"🔄 Exportando DataFrame final para: {output_path}")
final_df.to_csv(output_path, index=False, encoding='utf-8')
print("✅ Exportação concluída com sucesso!")

# 5. Exibir uma amostra do arquivo final
print("\n🎉 Processo concluído! Abaixo uma amostra do arquivo final gerado:")
print(final_df[['review_id', 'review_score', 'cleaned_text', 'Sentiment_Class', 'Sentiment_Polarity']].head())
print("\n📊 Contagem final de classes no arquivo exportado:")
print(final_df['Sentiment_Class'].value_counts())

🚀 Iniciando instalação de bibliotecas...
✅ Bibliotecas instaladas com sucesso!
✅ Módulos importados e ambiente configurado.

🔄 Montando Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔄 Carregando dataset de '/content/drive/MyDrive/Colab Notebooks/Olist/olist_order_reviews_dataset.csv'...
✅ DataFrame 'reviews_df' carregado com sucesso!

📋 Informações do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6  

  0%|          | 0/99224 [00:00<?, ?it/s]


✅ Pré-processamento concluído.
Total de reviews no dataset: 99224
Reviews com texto para análise: 42568

👀 Amostra do texto concatenado e limpo:
                                     review_full_text  \
3             . Recebi bem antes do prazo estipulado.   
4   . Parabéns lojas lannister adorei comprar pela...   
9   recomendo. aparelho eficiente. no site a marca...   
12      . Mas um pouco ,travando...pelo valor ta Boa.   
15  Super recomendo. Vendedor confiável, produto o...   

                                         cleaned_text  
3                recebi bem antes do prazo estipulado  
4   parabns lojas lannister adorei comprar pela in...  
9   recomendo aparelho eficiente no site a marca d...  
12             mas um pouco travandopelo valor ta boa  
15  super recomendo vendedor confivel produto ok e...  

🧠 Carregando o modelo de sentimento para Português...
2025-06-04 23:36:16,300 loading file PORTULAN/XLMR_Sentiment_Portuguese
❌ ERRO CRÍTICO: Falha ao carregar o modelo Flair

Extraindo resultados:   0%|          | 0/42568 [00:00<?, ?it/s]


✅ Análise de sentimento concluída!

📊 Contagem das classes de sentimento:
Sentiment_Class
NEGATIVE    21608
POSITIVE    20960
Name: count, dtype: int64

👀 Amostra dos resultados:
                                         cleaned_text Sentiment_Class  \
3                recebi bem antes do prazo estipulado        POSITIVE   
4   parabns lojas lannister adorei comprar pela in...        POSITIVE   
9   recomendo aparelho eficiente no site a marca d...        POSITIVE   
12             mas um pouco travandopelo valor ta boa        POSITIVE   
15  super recomendo vendedor confivel produto ok e...        POSITIVE   

    Sentiment_Polarity  
3                  0.0  
4                  0.0  
9                  0.0  
12                 0.0  
15                 0.0  

💾 Finalizando e preparando para exportação...
🔄 Exportando DataFrame final para: /content/drive/MyDrive/Colab Notebooks/Olist/olist_reviews_com_sentimento.csv
✅ Exportação concluída com sucesso!

🎉 Processo concluído! Abaixo uma a